<a href="https://colab.research.google.com/github/twaldburger/flood475/blob/master/geo475_flood_prediction_in_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geo475 - Flood prediction in Google Earth Engine

Go through the notebook and run the cells. Try to understand the code and the overall approach.  
Feel free to update code where you see room for improvement and do not hesitate to ask questions.

There are several tasks in the notebook, which are intended to trigger exploration and discussions. Please try to solve them but do not spend too much time on them. The goal is to understand the process and the code - solving all the tasks is secondary. 

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Tasks are marked like this.
</div> 

The notebook with answers to the tasks can be found [here](https://github.com/twaldburger/flood475/blob/master/geo475_flood_prediction_in_gee_master.ipynb).

---
## Setup

We use the *[Earth Engine](https://developers.google.com/earth-engine/guides/python_install)* and the *[geemap](https://geemap.org/)* Python libraries to interact with GEE. *geemap* was originally created because the offical *ee* library had relatively little documentation and limited functionality while *geemap* enables users to easily analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.

In this first code cell, we are importing our dependencies, setting some global variables and authenticating with GEE. All dependencies are already pre-installed in Colab.

In [ ]:
import ee
import geemap
import geemap.colormaps as cm
import time

## set some parameters, please update with your project id
PROJECT_ID = '' # your GEE project id
SAMPLE_SIZE = 100 # number of training locations per flood event and class
SEED = 3414 # for reproducible results

## connect to GEE
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

---
## Data exploration and visualization

GEE is built around two fundamental classes to represent raster and vector data: *[ee.Image](https://developers.google.com/earth-engine/apidocs/ee-image)* represents a single raster image while *[ee.Feature](https://developers.google.com/earth-engine/apidocs/ee-feature)* represents a geometry. Multiple images or geometries are represented as *[ee.ImageCollection](https://developers.google.com/earth-engine/apidocs/ee-imagecollection)* or as *[ee.FeatureCollection](https://developers.google.com/earth-engine/apidocs/ee-featurecollection)*.

The main catalogs for GEE data are the official [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets) and the community-maintained [awesome-gee-community-catalog](https://gee-community-catalog.org/). The Earth Engine Data Catalog stores over 90 petabytes of data and over 1'000 datasets while the awesome-gee-community-catalog stores about 500 terabytes of data and around 4'000 datasets. One of the big benefits of this data is, that they are curated, meaning that they have already been pre-processed and ingested and are ready for analysis. All we need to do is to select and use them.

In the cell below, we select a few input datasets that we will be using to sample our training data for the model training.

In [ ]:
## define the datasets from which to derive the input features
globalFlood = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
        .select('DEM')
landcover = ee.ImageCollection('ESA/WorldCover/v200')
hydro = ee.Image('MERIT/Hydro/v1_0_1')
prec = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
         .select('total_precipitation')
runoffPotential = ee.Image('projects/sat-io/open-datasets/HiHydroSoilv2_0/Hydrologic_Soil_Group_250m') \
                    .remap([1, 2, 3, 4, 14, 24, 34], [1, 2, 3, 4, 1, 3, 4]) \
                    .select('remapped')

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Check out the <a href="https://developers.google.com/earth-engine/datasets" target="_blank">Earth Engine Data Catalog</a> and the <a href="https://gee-community-catalog.org/" target="_blank">awesome-gee-community-catalog</a>.
  <ul style="margin-top: 10px; margin-bottom: 0;">
    <li>How are they structured?</li>
    <li>What information do they provide?</li>
    <li>Can you find the datasets we intend to use?</li>
    <li>What are the spatial and temporal resolution of "our" datasets?</li>
  </ul>
</div>

After checking on our input data in the catalogs, we want to visualize them on a map. *[geemap.Map](https://geemap.org/geemap/#geemap.geemap.Map)* provides a class for interactive mapping of GEE data in a Jupyter Notebook.

In the cell below, initialize a map with some basemaps and add the our historic flood layer. To correctly visualize the flood footprints, we need to ensure that we only include temporary inundation (the flood event) and not the permanent water bodies. To achieve this, we define a function *subtractPermanentWater*, which we will then run over every single image in our *ee.ImageCollection* of global flood events.

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Try to understand the logic in the <em>subtractPermanentWater</em> function in the cell below.
  <ol style="margin-top: 10px; margin-bottom: 0;">
    <li>How exactly does it remove the permanent water bodies from the floods? </li>
    <li>Why are we only using classes from the <em>ee</em> library and not some other Python library such as <em>numpy</em> or <em>xarray</em>?</li>
    <li>What do we achieve by calling <code>.sum()</code> in <code>flood = globalFlood.map(subtractPermanentWater).sum()</code> in row 6?</li>
  </ol>
</div>

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Run the cell below and check out the map.
  <ol>
    <li>Try adding a few more of our input datasets to the map as additional layers so you can explore them visually. We are interested in elevation, landcover, upstream drainage area, precipitation and runoff potential. You can find the geemap documentation <a href="https://geemap.org/">here</a>. If you are blocked, you can check my solution in <a href="https://github.com/twaldburger/flood475/blob/master/geo475_flood_prediction_in_gee_master.ipynb">this notebook</a>.</li>
    <li>How are the floods distributed? Are there geographies without any or with only a few floods?</li>
  </ol>
</div>

In [ ]:
## subtract the permanent water bodies from the flooded areas
def subtractPermanentWater(img):
  flood = img.select('flooded')
  perm = img.select('jrc_perm_water')
  return flood.multiply(perm.eq(0))
flood = globalFlood.map(subtractPermanentWater).sum()

## initialize map and add some basemaps
# see https://stackoverflow.com/a/33023651 for Google basemap list
Map = geemap.Map(center=[27, -81], zoom=7, basemap='CartoDB.DarkMatter')
Map.add_basemap('CartoDB.Positron', show=False)
Map.add_tile_layer("https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}", name="Google.Roadmap", attribution="Google", shown=False)
Map.add_tile_layer("https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}", name="Google.Satellite", attribution="Google", shown=False)

## add historic floods
flood_vis = {'min':0, 'max':10, 'palette':cm.palettes.Blues}
Map.add_layer(flood.selfMask(), flood_vis, 'Historic floods')
Map.add_colorbar(flood_vis, label="Number of floods", layer_name="Historic floods")

## display map
Map

---
## Training dataset

We are now somewhat familiar with out input data that we want to use for model training. However, the data are different raster datasets with different band information and various resolutions.

In the next cell, we define the logic to create a point dataset where every point is enriched with the corresponding pixel values from our input data. For this, we define two helper functions *pointQuery* and *removeProperty*, which we then use in our main function *createSample*.


<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Run the cell below and try to understand the code.
    <ol>
        <li>What is the general logic in <em>createSample</em>?</li>
        <li>Why are we using <em>ee.Image.stratifiedSample</em> in line 70 instead of using the much faster
            <em>ee.Image.Sample</em> method?</li>
        <li>Why are we using the complicated GEE methods in lines 65-76 and 90-94 instead of simply using plain Python?</li>
        <li>What does <em>ee.FeatureCollection.map</em> do? Why do we not just write a simple for-loop instead?</li>
        <li>Why do we normalize the elevation values?</li>
    </ol>
</div>

In [ ]:
def pointQuery(fc:ee.FeatureCollection, img:ee.Image, prop:str) -> ee.FeatureCollection:
  """
  Returns pixel values at locations using a feature collection and an image.

  Parameters
  ----------
  fc: : ee.FeatureCollection
    Collection of points at which to query the image.
  img : ee.Image
    The image to query.
  prop : str
    Name of new property to hold the query results.

  Returns
  -------
  ee.FeatureCollection
    Input FeatureCollection with lookup values added as new property.
  """
  fc = img.reduceRegions(collection=fc, reducer=ee.Reducer.first())
  return fc.map(lambda feat: feat.set(prop, feat.get('first')))


def removeProperty(fc:ee.FeatureCollection, prop:str) -> ee.FeatureCollection:
  """
  Removes a property by name from a feature collection.

  Parameters
  ----------
  fc : ee.FeatureCollection
    Collection from which to remove the property.
  prop : str
    Property to remove.

  Returns
  -------
  ee.FeatureCollection
    Input collection without the removed property.
  """
  selectProperties = fc.propertyNames().filter(ee.Filter.neq('item', prop))
  return fc.select(selectProperties)


def createSample(img:ee.Image) -> ee.FeatureCollection:
  """
  Samples training dataset for a single flood image.

  Parameters
  ----------
  img : ee.Image
    Input image.

  Returns
  -------
  ee.FeatureCollection
    Sampled and enriched training data.
  """

  ## subtract permanent water bodies from flooded areas
  permanent = img.select('jrc_perm_water')
  water = img.select('flooded')
  flooded = water.subtract(permanent).gt(0)

  ## get the total and maximum precipitation over 14 days prior to the event end date
  end = img.getNumber('system:time_end')
  start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000
  precSum = prec.filter(ee.Filter.date(start, end)).sum()
  precMax = prec.filter(ee.Filter.date(start, end)).max()

  ## sample equal number of flooded and non-flooded points
  sample = flooded.stratifiedSample(numPoints=SAMPLE_SIZE, classBand='flooded', geometries=True)

  ## add image id in case we want to join the event metadata later
  sample = sample.map(lambda x: x.set('eventId', img.get('system:index')))

  ## enrich sample by running point lookups on multiple datasets
  sample = pointQuery(sample, dem.mosaic(), 'demElevationAbs')
  sample = pointQuery(sample, ee.Terrain.aspect(dem.mosaic()), 'demAspect')
  sample = pointQuery(sample, ee.Terrain.slope(dem.mosaic()), 'demSlope')
  sample = pointQuery(sample, landcover.first(), 'landcover')
  sample = pointQuery(sample, hydro.select('upa'), 'upa')
  sample = pointQuery(sample, runoffPotential, 'runoffPot')
  sample = pointQuery(sample, precSum, 'precSum')
  sample = pointQuery(sample, precMax, 'precMax')

  ## remove first-property
  sample = sample.map(lambda feat: removeProperty(feat, 'first'))

  ## normalize elevation
  elevationRange = dem.mosaic().reduceRegion(geometry=img.geometry(), reducer=ee.Reducer.minMax())
  min = ee.Number(elevationRange.get('DEM_min'))
  max = ee.Number(elevationRange.get('DEM_max'))
  def normalizeElevation(feat:ee.Feature) -> ee.Feature:
    return feat.set('demElevationNorm', (ee.Number(feat.get('demElevationAbs')).subtract(min)).divide(max.subtract(min)))
  sample = sample.filter(ee.Filter.notNull(ee.List(['demElevationAbs']))).map(normalizeElevation)

  return sample

The last cell defined the functions to process flood event images and extract training samples with enriched features. We now want to execute those functions as a batch job and export the result to a GEE asset.

The next cell defines the export task.

In [ ]:
## create task to enrich sample and store the result as an asset
properties = ['demAspect', 'demElevationAbs', 'demElevationNorm', 'demSlope', 'eventId', 'flooded', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa', '.geo']
sample = globalFlood.map(createSample).flatten()
sample = sample.filter(ee.Filter.notNull(properties)).distinct(properties)
task = ee.batch.Export.table.toAsset(sample, description='flood475-sampling', assetId=f"projects/{PROJECT_ID}/assets/flood475_sample")

And this cell actually runs it. It is run on GEE and there are several ways to monitor it. One way is to constantly request the task status from our notebook (the commented part in the cell below). However, this also blocks our notebook so we rather choose one of the other ways to monitor progress:
- [GEE Code editor](https://code.earthengine.google.com/)
- [Task Manager](https://code.earthengine.google.com/tasks)
- [Tasks Page in the Cloud Console](https://console.cloud.google.com/earth-engine/tasks?project=ee-timwaldburger-flood475)

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Run the cell below and check if your task is running via one of the different methods.
</div>

In [ ]:
## run and monitor the task
task.start()
# while task.active():
#   ts = task.status()
#   if ts['start_timestamp_ms']>0:
#     s = round((ts['update_timestamp_ms']-ts['start_timestamp_ms'])/1000)
#   else:
#     s = round((ts['update_timestamp_ms']-ts['creation_timestamp_ms'])/1000)
#   print(f"task '{ts['description']}' is {ts['state']} for {s} seconds")
#   time.sleep(60)
# task.status()

Let us now import our exported assed and visualize our training data. Depending on your sample size, training may take up to 30 minutes or more. You can therefore load the training data I created previously by running the cell below:

In [ ]:
## import the pre-created training dataset
sample = ee.FeatureCollection(f"projects/ee-timwaldburger-flood475/assets/flood475_sample")

## import your training dataset
# sample = ee.FeatureCollection(f"projects/{PROJECT_ID}/assets/flood475_sample")

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Add the training locations to the map we created above. If you are blocked, you can check my solution in <a href="https://github.com/twaldburger/flood475/blob/master/geo475_flood_prediction_in_gee_master.ipynb">this notebook</a>.
</div>

---
## Model training

We have now prepared our training data and are ready to train our model. To keep dependencies as simple as possible, we chose a [random forest](https://www.ibm.com/think/topics/random-forest), which is available within GEE from *[ee.Classifier.smileRandomForest](https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest)*. We train the model using only 70 % of our test data and reserve the other 30 % for validation.

In [ ]:
## import the training dataset
sample = ee.FeatureCollection(f"projects/{PROJECT_ID}/assets/flood475_sample")
# sample = ee.FeatureCollection(f"projects/ee-timwaldburger-flood475/assets/flood475_sample")

## partition into 70% training and 30% validation samples
sample = sample.randomColumn('random', seed=SEED)
training = sample.filter(ee.Filter.lt('random', 0.7))
validation = sample.filter(ee.Filter.gte('random', 0.7))

## train a random forest
properties = ['demAspect', 'demElevationNorm', 'demSlope', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa']
randomForest = ee.Classifier.smileRandomForest(10).setOutputMode('CLASSIFICATION')
classifier = randomForest.train(
    features=training,
    classProperty='flooded',
    inputProperties=properties
)

The cell below calcualtes and prints some accuracy metrics on the training and the validation dataset:

In [ ]:
# accuracy on training set
trainConfusionMatrix = classifier.confusionMatrix()
trainFscores = trainConfusionMatrix.fscore().getInfo()
print(f"train accuracy: {trainConfusionMatrix.accuracy().getInfo():.3f}")
print(f"train f-score non-flooded: {trainFscores[0]:.3f}")
print(f"train f-score flooded: {trainFscores[1]:.3f}")

# accuracy on validation set
testConfusionMatrix = validation.classify(classifier).errorMatrix('flooded', 'classification')
testFscores = testConfusionMatrix.fscore().getInfo()
print(f"validation accuracy: {testConfusionMatrix.accuracy().getInfo():.3f}")
print(f"validation f-score non-flooded: {testFscores[0]:.3f}")
print(f"validation f-score flooded: {testFscores[1]:.3f}")

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Check the metrics returned by the cell above.
    <ol>
        <li>What do accuracy and F1-Score describe?</li>
        <li>Do you think the model performs well given those results?</li>
    </ol>
</div>

Let us try once more by setting some parameters to address the overfitting issue. We will not worry to much about class imbalance since we have used stratified sampling to ensure same number of test locations per class.

In [ ]:
## train another random forest with parameters to control overfitting
properties = ['demAspect', 'demElevationNorm', 'demSlope', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa']
randomForest = ee.Classifier.smileRandomForest(
    numberOfTrees=50,             # sufficient ensemble size
    maxNodes=32,                  # controls tree complexity (prevents deep memorization)
    minLeafPopulation=5,          # ensures leaf nodes have enough samples (improves robustness)
    bagFraction=0.63,             # recommended fraction for sampling
).setOutputMode('CLASSIFICATION')
classifier = randomForest.train(
    features=training,
    classProperty='flooded',
    inputProperties=properties
)

# accuracy on training set
trainConfusionMatrix = classifier.confusionMatrix()
trainFscores = trainConfusionMatrix.fscore().getInfo()
print(f"train accuracy: {trainConfusionMatrix.accuracy().getInfo():.3f}")
print(f"train f-score non-flooded: {trainFscores[0]:.3f}")
print(f"train f-score flooded: {trainFscores[1]:.3f}")

# accuracy on validation set
testConfusionMatrix = validation.classify(classifier).errorMatrix('flooded', 'classification')
testFscores = testConfusionMatrix.fscore().getInfo()
print(f"validation accuracy: {testConfusionMatrix.accuracy().getInfo():.3f}")
print(f"validation f-score non-flooded: {testFscores[0]:.3f}")
print(f"validation f-score flooded: {testFscores[1]:.3f}")

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Check the metrics returned by the new model.
    <ol>
        <li>Could we improve our model?</li>
    </ol>
</div>

---
## Prediction

We know that our model is flawed, but let's still go ahead and run some predictions.

The cell below defined the *predict* function, which takes a region of interest and a classifier (our random forest) as input.

<div style="padding: 15px; border-left: 6px solid #f38a21ff;">
  <strong style="color: #f38a21ff;">Task:</strong> Try to understand the code in the cell below.
    <ol>
        <li>What does the <em>predict</em> function do exactly?</li>
    </ol>
</div>

<div style="padding: 15px; border-left: 6px solid #1fbd5bff;">
<strong style="color: #1fbd5bff;">Solution:</strong>
  <ol>
    <li>The <em>predict </em>function creates a composite image by combining all the datasets we used for model training. It normalizes the elevation and aggregates precipitation. It then classifies every pixel in the composite using the pre-trained model.</li>
  </ol>
</div>

In [ ]:
def predict(roi: ee.Geometry, classifier: ee.Classifier) -> ee.Image:
  """
  Predict flood probability for a given region of interest.

  Parameters
  ----------
  roi : ee.Geometry
    Region of interest.
  classifier : ee.Classifier
    Trained classifier.

  Returns
  -------
  ee.Image
    Flood probabilities.
  """

  ## normalize elevation
  elevationRange = dem.mosaic().reduceRegion(geometry=roi, reducer=ee.Reducer.minMax(), scale=30, bestEffort=True)
  min = ee.Number(elevationRange.get('DEM_min'))
  max = ee.Number(elevationRange.get('DEM_max'))

  ## calculate start and end data for precipitation data aggregation
  end = ee.Date('2024-10-31T00:00:00').millis()
  start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000

  ## create composite of all relevant datasets
  composite = ee.Image.cat(
      ee.Terrain.aspect(dem.mosaic()),
      dem.mosaic().unitScale(min, max),
      ee.Terrain.slope(dem.mosaic()),
      landcover.first(),
      prec.filter(ee.Filter.date(start, end)).max(),
      prec.filter(ee.Filter.date(start, end)).sum(),
      runoffPotential,
      hydro.select('upa')
  ).rename(properties)

  ## classify the composite
  classifier = classifier.setOutputMode('MULTIPROBABILITY')
  classified = composite.classify(classifier).clip(roi)
  probabilities = classified.arrayFlatten([['non-flooded', 'flooded']])
  probability = probabilities.select('flooded')

  return probability

Finally, we are ready to make predictions. For this, we call our *predict* function for a given region of interest using our random forest.

<div style="padding: 15px; border-left: 6px solid #e12525ff;">
  <strong style="color: #e12525ff;">Warning:</strong> The cell below uses the map bounds of our map where we explored the data as the extent for which to compute predictions. Make sure to zoom to a relatively small area to avoid long running computations.
</div>

In [ ]:
## run prediction for current map bounds
probability = predict(roi=ee.Geometry.BBox(*Map.getBounds()), classifier=classifier)

## update mask to exclude permanent water
permanentWater = globalFlood.select('jrc_perm_water').mosaic()
probability = probability.updateMask(permanentWater.neq(1))

## update the map
probability_vis = {'min':0, 'max':1, 'palette':cm.palettes.cividis}
Map.addLayer(probability.selfMask(), probability_vis, 'Flooded probability')
Map.add_colorbar(probability_vis, label="Flooded probability", layer_name="Flooded probability", font_size=9)
Map

---
## Feedback

Providing a 2 minute feedback would help me to improve the exercise.
Please run the cell below to display a Google Form where you can provide your feedback. I will not collect your mail address so the feedback is anonymous.

In [ ]:
%%html
<iframe src="https://docs.google.com/forms/d/e/1FAIpQLScg8j6ORkqgWw4QEHpkeOy2PxYKSdgop3PPvaA1_WT54igFIA/viewform?embedded=true" width="640" height="1304" frameborder="0" marginheight="0" marginwidth="0">Loading…</iframe>

---
## Bonus: run country-scale prediction
The code below runs the trained model for all of Switzerland at 30 m resolution and exports the result to GEE. It runs in about 20 minutes. No need to run in the lab, but feel free to try it out and play around with different models, geographies and settings.

In [ ]:
## get country shape
countries = ee.FeatureCollection('WM/geoLab/geoBoundaries/600/ADM0')
ch = countries.filterMetadata('shapeName', 'equals', 'Switzerland')

## run prediction for Switzerland
flooded_prob = predict(roi=ch.geometry(), classifier=classifier)

## update the map
flooded_prob_vis = {'min':0, 'max':1, 'palette':cm.palettes.cividis}
Map.addLayer(flooded_prob.selfMask(), flooded_prob_vis, 'Flooded probability')
Map.add_colorbar(flooded_prob_vis, label="CH flooded probability", layer_name="CH flooded probability", font_size=9)
Map

In [ ]:
## create export task
task = ee.batch.Export.image.toAsset(
  flooded_prob,
  description='flood475-ch-prediction',
  assetId=f"projects/{PROJECT_ID}/assets/flood475_ch_prediction",
  scale=30,
  maxPixels=500_000_000
)

## run and monitor the task
task.start()
while task.active():
  ts = task.status()
  if ts['start_timestamp_ms']>0:
    s = round((ts['update_timestamp_ms']-ts['start_timestamp_ms'])/1000)
  else:
    s = round((ts['update_timestamp_ms']-ts['creation_timestamp_ms'])/1000)
  print(f"task '{ts['description']}' is {ts['state']} for {s} seconds")
  time.sleep(60)
task.status()